# Halvor -- problem d
## Cross-validation as resampling techniques, adding more complexity

## Why resampling methods:
Before we proceed, we need to rethink what we have been doing. In our eager to fit the data, we have omitted several important elements in our regression analysis. In what follows we will

1.look at statistical properties, including a discussion of mean values, variance and the so-called bias-variance tradeoff

2.introduce resampling techniques like cross-validation, bootstrapping and jackknife and more
and discuss how to select a given model (one of the difficult parts in machine learning).

Resampling methods are an indispensable tool in modern statistics. They involve repeatedly drawing samples from a training set and refitting a model of interest on each sample in order to obtain additional information about the fitted model. For example, in order to estimate the variability of a linear regression fit, we can repeatedly draw different samples from the training data, fit a linear regression to each new sample, and then examine the extent to which the resulting fits differ. Such an approach may allow us to obtain information that would not be available from fitting the model only once using the original training sample

Our simulations can be treated as computer experiments. This is particularly the case for Monte Carlo methods which are widely used in statistical analyses.

The results can be analysed with the same statistical tools as we would use when analysing experimental data.

As in all experiments, we are looking for expectation values and an estimate of how accurate they are, i.e., possible sources for errors.

### two resampeling methods:
a.bootstrap - problem c)

b.cross validation 

# Cross validation:

cross-validation can be used to estimate the test error associated with a given statistical learning method in order to evaluate its performance, or to select the appropriate level of flexibility. The process of evaluating a model’s performance is known as model assessment, whereas the process of selecting the proper level of flexibility for a model is known as model selection.

## Various steps in cross-validation

When the repetitive splitting of the data set is done randomly, samples may accidently end up in a fast majority of the splits in either training or test set. Such samples may have an unbalanced influence on either model building or prediction evaluation. 

To avoid this k-fold cross-validation structures the data splitting. 
The samples are divided into k, more or less equally sized exhaustive and mutually exclusive subsets. In turn (at each split) one of these subsets plays the role of the test set while the union of the remaining subsets constitutes the training set. 

Such a splitting warrants a balanced representation of each sample in both training and test set over the splits. Still the division into the k subsets involves a degree of randomness. 

This may be fully excluded when choosing k=n. This particular case is referred to as leave-one-out cross-validation (LOOCV).

### Cross-validation in brief
For the various values of k

1.shuffle the dataset randomly.

2.Split the dataset into k groups.

3.For each unique group:

    a.Decide which group to use as set for test data
    
    b.Take the remaining groups as a training data set
    
    c.Fit a model on the training set and evaluate it on the test set
    
    d.Retain the evaluation score and discard the model
    
4.Summarize the model using the sample of model evaluation scores

# Problem text:
--------------------------
The aim here is to write your own code for another widely popular resampling technique, the so-called cross-validation method. Again, before you start with cross-validation approach, you should scale your data if you think this is needed.

Implement the k
-fold cross-validation algorithm (write your own code) and evaluate again the MSE function resulting from the test folds. You can compare your own code with that from Scikit-Learn if needed.

Compare the MSE you get from your cross-validation code with the one you got from your bootstrap code. Comment your results. Try 5−10
 folds. You can also compare your own cross-validation code with the one provided by Scikit-Learn.
------------------------

# Problem Solution:

In [359]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [339]:
#from reggresion_tools:
def create_X_polynomial(x, y, n):
    if len(x.shape) > 1:
        x = np.ravel(x)
        y = np.ravel(y)

    N = len(x)
    l = int((n+1)*(n+2)/2)      # Number of elements in beta
    X = np.ones((N,l))

    for i in range(1,n+1):
        q = int((i)*(i+1)/2)
        for k in range(i+1):
            X[:,q+k] = (x**(i-k))*(y**k)

    return X

def linreg_polynomial(x, y, z, n):
    X = create_X_polynomial(x, y, n)

    # Solving for beta
    beta = np.linalg.inv(X.T @ X) @ X.T @ z
    return beta

def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

## model Ridge and Lasso implementation:

In [441]:
#Ridge regression
def ridgereg_polynomial(x, y, z, degree, lambdan):
    #design matrix
    X = create_X_polynomial(x, y, degree)
    #create identity matrix
    I = np.eye(len(X.T), len(X.T))
    
    # Solving for beta
    beta_ridge = np.linalg.pinv(X.T@X + lambdan*I) @ X.T @ z
    return beta_ridge

#Lassso mit scikit-learn:
def lassoreg_polynomial(x, y, z, degree, lambdan):
    #design matrix
    X = create_X_polynomial(x, y, degree)
    
    #Lasso regression with scikit-learn 
    RegLasso = Lasso(lambdan)
    RegLasso.fit(X,z)
    beta_lasso = RegLasso.coef_
    return beta_lasso

In [442]:
np.random.seed(244) #The same as in task b)

In [443]:
#franke fuction from task b)
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

In [464]:
# creating datapoints
x = np.np.random.random(40)) # + (1/5) * np.random.normal(0, 1, 25))
y = np.sort(np.random.random(40))
z = FrankeFunction(x, y)
print(z)

In [473]:
def cross_validation(k_deg_fold, x, y, z, model_fit=linreg_polynomial, degree=2, lambdan=0):
    #step 1: shuffle datasets randomly using np.random.permutation(len(x)):
    assert len(x) == len(z) == len(y)
    p = np.random.permutation(len(x))
    x, y, z = x[p], y[p], z[p]
    
    #step 2: split the data in k groups with numpy.array_split
    x = np.array_split(x, k_deg_fold); 
    y = np.array_split(y, k_deg_fold); 
    z = np.array_split(z, k_deg_fold)

    # array to keep track of MSE for each test-group
    MSE_array = np.zeros((k_deg_fold))
    
    #step 3:
    for i in range(k_deg_fold):
        #a) pick one group to be test data
        x_test, y_test, z_test = x[i], y[i], z[i]
        
        #b) take remaining groupe as train data
        x_train = np.ndarray.flatten(np.array(x[:i] + x[i+1:]))
        y_train = np.ndarray.flatten(np.array(y[:i] + y[i+1:]))
        z_train = np.ndarray.flatten(np.array(z[:i] + z[i+1:]))
        
        #X_train = create_X_polynomial(x_train, y_train, 2)
        
        #c) fit model to train data
        if model_fit == linreg_polynomial:
            beta = model_fit(x_train, y_train, z_train, degree)
        elif model_fit == ridgereg_polynomial or model_fit == lassoreg_polynomial:
            beta = model_fit(x_train, y_train, z_train, degree, lambdan)
        
        #d) evaluate model and save score-value
        X_test = create_X_polynomial(x_test, y_test, degree)
        z_tilde = X_test @ beta 
        MSE_array[i] = MSE(z_test, z_tilde)
        
    return MSE_array

In [474]:
np.random.seed(244)
cross_validation(10, x, y, z, model_fit=linreg_polynomial)

array([0.00346698, 0.01052689, 0.01128705, 0.03093545, 0.00602354,
       0.00240833, 0.00749399, 0.00863021, 0.00134337, 0.00507218])

In [475]:
cross_validation(10, x, y, z, model_fit=ridgereg_polynomial, lambdan=0.5)

array([0.0102417 , 0.0064399 , 0.02432365, 0.00538653, 0.00500456,
       0.01907118, 0.003358  , 0.00522926, 0.01152256, 0.07964081])

In [476]:
cross_validation(10, x, y, z, model_fit=lassoreg_polynomial, lambdan=0.5)

array([0.19462191, 0.08821231, 0.49326773, 0.16798021, 0.1149552 ,
       0.15976573, 0.0778772 , 0.38857618, 0.0347604 , 0.14858592])